# Sparkling Emoticana

In [ ]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import scipy.io
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display

%matplotlib inline

In [ ]:
sc

## 1. denseNet

In [ ]:
class denseNet1() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def dense_block(self, X_input, num_filter, num_layer, name) :
        with tf.variable_scope(name) :
            layer_lst = [X_input]
            concat = X_input
            
            for idx in range(num_layer) :
                bn1 = tf.layers.batch_normalization(inputs=concat)
                relu1 = tf.nn.leaky_relu(bn1)
                conv1 = tf.layers.conv2d(inputs=relu1, filters=num_filter*4, kernel_size=[1,1], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  
                    
                bn2 = tf.layers.batch_normalization(inputs=conv1)
                relu2 = tf.nn.leaky_relu(bn2)
                conv2 = tf.layers.conv2d(inputs=relu2, filters=num_filter, kernel_size=[3,3], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  

                layer_lst.append(conv2)
                concat = tf.concat(layer_lst, axis=3)
                
            return concat
        
    def transition_block(self, X_input, theta, name) :
        with tf.variable_scope(name) :
            depth = X_input.get_shape().as_list()[3]
            
            bn = tf.layers.batch_normalization(inputs=X_input)
            relu = tf.nn.leaky_relu(bn)
            conv = tf.layers.conv2d(inputs=relu, filters=depth*theta, kernel_size=[1,1], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  
            pool = tf.layers.average_pooling2d(inputs=conv, pool_size=[2,2], strides=2,)     
            
            return pool
        
    def build(self, k_init=40, k=12) :
        with tf.variable_scope(self.name) :
            ### Input
            #input : 128x126x1
            #output : 8
            self.X = tf.placeholder(tf.float32, [None, 128, 126, 1])
            self.Y = tf.placeholder(tf.float32, [None, 8])
            self.training = tf.placeholder(tf.bool)
            self.learning_rate = tf.placeholder(tf.float32)
            print(self.X.shape)

            ### Input Layer
            #input : 128x126x1
            #output : 64x63x40
            self.bn = tf.layers.batch_normalization(self.X)
            self.conv = tf.layers.conv2d(inputs=self.bn, filters=k_init, kernel_size=[3,3], strides=2, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.relu = tf.nn.leaky_relu(self.conv)
            
            print(self.conv.shape)
            print("-"*10)
            
        ### Hidden Layer
        #input : 64x63x40
        #output : 32x31x84
        self.dense1 = self.dense_block(self.relu, k, 8, "dense1")
        self.transition1 = self.transition_block(self.dense1, 0.5, name="transition1")
        print(self.dense1.shape)
        print(self.transition1.shape)
        print("-"*10)
        
        #input : 32x31x84
        #output : 16x15x138
        self.dense2 = self.dense_block(self.transition1, k, 12, "dense2")
        self.transition2 = self.transition_block(self.dense2, 0.5, name="transition2")
        print(self.dense2.shape)
        print(self.transition2.shape)
        print("-"*10)
        
            
        #input : 16x15x138
        #output : 8x7x165
        self.dense3 = self.dense_block(self.transition2, k, 12, "dense3")
        self.transition3 = self.transition_block(self.dense3, 0.5, name="transition3")
        print(self.dense3.shape)
        print(self.transition3.shape)
        print("-"*10)
        
        #input : 8x7x317
        #output : 8x7x165
        self.dense4 = self.dense_block(self.transition3, k, 8, "dense4")
        print(self.dense4.shape)
        print("-"*10)
        
        #input : 8x7x165
        #output : 1x1x165
        with tf.variable_scope("global_avg_pooling") :
            self.bn = tf.layers.batch_normalization(self.dense4)
            self.relu = tf.nn.leaky_relu(self.bn)
            self.global_avg_pooling = tf.reduce_mean(self.relu, [1, 2], keep_dims=True)
            print(self.global_avg_pooling.shape)
            print("-"*10)
        
        with tf.variable_scope("fully_connected") :
            ###Output Layer
            #input : 1x1x165
            #ouput : 8
            shape = self.global_avg_pooling.get_shape().as_list()
            dimension = shape[1] * shape[2] * shape[3]
            self.flat = tf.reshape(self.global_avg_pooling, shape=[-1, dimension])

            self.fc = tf.layers.dense(inputs=self.flat, units=8, kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.logits = self.fc

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        feed_dict={self.X: x_test, self.training: training}
        
        return self.sess.run(self.logits, feed_dict=feed_dict)

    def get_accuracy(self, x_test, y_test, training=False):
        feed_dict={self.X: x_test,self.Y: y_test, self.training: training}
        
        return self.sess.run(self.accuracy, feed_dict=feed_dict)

    def train(self, x_data, y_data, learning_rate, training=True):
        feed_dict={self.X: x_data, self.Y: y_data, self.learning_rate: learning_rate, self.training: training}
        
        return self.sess.run([self.cost, self.optimizer], feed_dict=feed_dict)
    
    def evaluate(self, X_input, Y_input, batch_size=None, training=False):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: training}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc
    
    def save(self) :
        saver = tf.train.Saver(ver)
        save_path = saver.save(self.sess, "denseNet_" + str(ver) + ".ckpt")
        
        print("Model saved in path: %s" % save_path)
                 

In [ ]:
class denseNet2() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def dense_block(self, X_input, num_filter, num_layer, name) :
        with tf.variable_scope(name) :
            layer_lst = [X_input]
            concat = X_input
            
            for idx in range(num_layer) :
                bn1 = tf.layers.batch_normalization(inputs=concat)
                relu1 = tf.nn.leaky_relu(bn1)
                conv1 = tf.layers.conv2d(inputs=relu1, filters=num_filter*4, kernel_size=[1,1], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  
                    
                bn2 = tf.layers.batch_normalization(inputs=conv1)
                relu2 = tf.nn.leaky_relu(bn2)
                conv2 = tf.layers.conv2d(inputs=relu2, filters=num_filter, kernel_size=[1,3], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  

                layer_lst.append(conv2)
                concat = tf.concat(layer_lst, axis=3)
                
            return concat
        
    def transition_block(self, X_input, theta, name) :
        with tf.variable_scope(name) :
            depth = X_input.get_shape().as_list()[3]
            
            bn = tf.layers.batch_normalization(inputs=X_input)
            relu = tf.nn.leaky_relu(bn)
            conv = tf.layers.conv2d(inputs=relu, filters=depth*theta, kernel_size=[1,1], strides=1, padding="SAME", kernel_initializer=tf.contrib.layers.xavier_initializer())  
            pool = tf.layers.average_pooling2d(inputs=conv, pool_size=[1,2], strides=2)     
            
            return pool
        
    def build(self, k_init=40, k=16) :
        with tf.variable_scope(self.name) :
            ### Input
            #input : 128x126x1
            #output : 8
            self.X = tf.placeholder(tf.float32, [None, 128, 126, 1])
            self.Y = tf.placeholder(tf.float32, [None, 8])
            self.training = tf.placeholder(tf.bool)
            self.learning_rate = tf.placeholder(tf.float32)
            print(self.X.shape)

            ### Input Layer
            #input : 128x126x1
            #output : 1x63x40
            self.bn = tf.layers.batch_normalization(self.X)
            self.conv = tf.layers.conv2d(inputs=self.bn, filters=k_init, kernel_size=[128,3], strides=2, kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.relu = tf.nn.leaky_relu(self.conv)
            
            print(self.conv.shape)
            print("-"*10)
            
        ### Hidden Layer
        #input : 1x63x40
        #output : 1x31x84
        self.dense1 = self.dense_block(self.relu, k, 8, "dense1")
        self.transition1 = self.transition_block(self.dense1, 0.5, name="transition1")
        print(self.dense1.shape)
        print(self.transition1.shape)
        print("-"*10)
        
        #input : 1x31x84
        #output : 1x15x138
        self.dense2 = self.dense_block(self.transition1, k, 12, "dense2")
        self.transition2 = self.transition_block(self.dense2, 0.5, name="transition2")
        print(self.dense2.shape)
        print(self.transition2.shape)
        print("-"*10)
        
            
        #input : 1x15x138
        #output : 1x7x165
        self.dense3 = self.dense_block(self.transition2, k, 12, "dense3")
        self.transition3 = self.transition_block(self.dense3, 0.5, name="transition3")
        print(self.dense3.shape)
        print(self.transition3.shape)
        print("-"*10)
        
        #input : 1x7x317
        #output : 1x7x165
        self.dense4 = self.dense_block(self.transition3, k, 8, "dense4")
        print(self.dense4.shape)
        print("-"*10)
        
        #input : 1x7x165
        #output : 1x1x165
        with tf.variable_scope("global_avg_pooling") :
            self.bn = tf.layers.batch_normalization(self.dense4)
            self.relu = tf.nn.leaky_relu(self.bn)
            self.global_avg_pooling = tf.reduce_mean(self.relu, [1, 2], keep_dims=True)
            print(self.global_avg_pooling.shape)
            print("-"*10)
        
        with tf.variable_scope("fully_connected") :
            ###Output Layer
            #input : 1x1x165
            #ouput : 8
            shape = self.global_avg_pooling.get_shape().as_list()
            dimension = shape[1] * shape[2] * shape[3]
            self.flat = tf.reshape(self.global_avg_pooling, shape=[-1, dimension])

            self.fc = tf.layers.dense(inputs=self.flat, units=8, kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.logits = self.fc

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        feed_dict={self.X: x_test, self.training: training}
        
        return self.sess.run(self.logits, feed_dict=feed_dict)

    def get_accuracy(self, x_test, y_test, training=False):
        feed_dict={self.X: x_test,self.Y: y_test, self.training: training}
        
        return self.sess.run(self.accuracy, feed_dict=feed_dict)

    def train(self, x_data, y_data, learning_rate, training=True):
        feed_dict={self.X: x_data, self.Y: y_data, self.learning_rate: learning_rate, self.training: training}
        
        return self.sess.run([self.cost, self.optimizer], feed_dict=feed_dict)
    
    def evaluate(self, X_input, Y_input, batch_size=None, training=False):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: training}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc
    
    def save(self) :
        saver = tf.train.Saver(ver)
        save_path = saver.save(self.sess, "denseNet_" + str(ver) + ".ckpt")
        
        print("Model saved in path: %s" % save_path)
                 

 <br></br>  <br></br>  <br></br>  <br></br>

## 2. dataset

In [ ]:
def load_mat_data(path) :
    file_lst = os.listdir(path)
    random.shuffle(file_lst)
    
    train = []
    valid = []
    test = []
    
    for file in file_lst :
        try : 
            emotion = int(file.split("-")[2])
            actor = int(file.split("-")[6].split(".")[0])
            spectrogram = scipy.io.loadmat(path+file)["S"]
        
            if actor in [1,2] :
                valid.append((spectrogram, emotion))
            elif actor in [3,4] :
                test.append((spectrogram, emotion))
            else :
                train.append((spectrogram, emotion))
                
        except :
            pass
    
    return file_lst, train, valid, test

In [ ]:
def load_wav_data(path) :
    file_lst = os.listdir(path)
    random.shuffle(file_lst)
    
    train = []
    valid = []
    test = []
    
    for file in file_lst :
        try : 
            y, sr = librosa.load(path+file)
            emotion = int(file.split("-")[2])
            actor = int(file.split("-")[6].split(".")[0])
        
            melspectrogram = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
        
            if actor in [1,2] :
                valid.append((melspectrogram, emotion))
            elif actor in [3,4] :
                test.append((melspectrogram, emotion))
            else :
                train.append((melspectrogram, emotion))
                
        except :
            pass
    
    return file_lst, train, valid, test

In [ ]:
def zero_padding(train, valid, test, num=528) :
    result = []
    
    for dataset in [train, valid, test] :
        zero = np.zeros([len(dataset), 1025, num])
        emotion_lst = []

        idx = 0
        for spectrogram, emotion in dataset:
            zero[idx, :, 0:len(spectrogram[0])] = spectrogram
            emotion_lst.append(emotion-1)
            idx += 1
            
        result.append((zero, emotion_lst))
        
    return result


In [ ]:
def cutting(train, valid, test, size=1025, num=276) :
    result = []
    half = int(num/2)
    
    for dataset in [train, valid, test] :
        zero = np.zeros([len(dataset), size, num])
        emotion_lst = []

        idx = 0
        for spectrogram, emotion in dataset:
            mid = int(spectrogram.shape[1]/2)
            zero[idx, :, 0:len(spectrogram[0])] = spectrogram[:, mid-half:mid+half]
            emotion_lst.append(emotion-1)
            idx += 1
            
        result.append((zero, emotion_lst))
        
    return result

In [ ]:
def onehot_encoding(data, num=8) :
    return np.eye(num)[data]

In [ ]:
# spectrogram from mat
"""
file_lst, train, valid, test = load_mat_data("data/mat/")
train, valid, test = cutting(train, valid, test)

train_data = train[0].reshape([-1, 1025, 276, 1])
train_label = onehot_encoding(train[1])

valid_data = valid[0].reshape([-1, 1025, 276, 1])
valid_label = onehot_encoding(valid[1])

test_data = test[0].reshape([-1, 1025, 276, 1])
test_label = onehot_encoding(test[1])

train = []
valid = []
test = []

print(len(train_data), train_data.shape, train_label.shape)
print(len(valid_data))
print(len(test_data))

print(file_lst[:5])
print(train_data[0])
plt.imshow(train_data[300].reshape(1025,276))
"""

In [ ]:
# melspectrogram from wav
file_lst, train, valid, test = load_wav_data("data/wav/")
cut_train, cut_valid, cut_test = cutting(train, valid, test, size =128 , num=126)

train_data = cut_train[0].reshape([-1, 128, 126, 1])
train_label = onehot_encoding(cut_train[1])

valid_data = cut_valid[0].reshape([-1, 128, 126, 1])
valid_label = onehot_encoding(cut_valid[1])

test_data = cut_test[0].reshape([-1, 128, 126, 1])
test_label = onehot_encoding(cut_test[1])

train = []
valid = []
test = []
cut_train = []
cut_valid = []
cut_test = []

In [ ]:
print(len(train_data), train_data.shape, train_label.shape)
print(len(valid_data))
print(len(test_data))
print(train_data[0][:5, :5])

plt.figure(figsize=(10, 4))
librosa.display.specshow(librosa.power_to_db(train_data[0].reshape(128,126), ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()

 <br></br>  <br></br>  <br></br>  <br></br>

## 3. train and test

In [ ]:
learning_rate = 0.01
training_epochs = 40
batch_size = 30

train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

In [ ]:
sess = tf.Session()

model = denseNet1(sess, "denseNet1")
model.build()

In [ ]:
sess.run(tf.global_variables_initializer())
print('Learning Started!')
print("")

# train model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(train_data) / batch_size)
    idx = 0
    if epoch == 1 :
        learning_rate = 0.005
    elif epoch == 3 :
        learning_rate = 0.003
    elif epoch == 5 :
        learning_rate = 0.002
    elif epoch == 10 :
        learning_rate = 0.001

    for i in range(total_batch):
        batch_xs, batch_ys = train_data[idx:idx+batch_size], train_label[idx:idx+batch_size]
        c, _ = model.train(batch_xs, batch_ys, learning_rate=learning_rate)
        avg_cost += c / total_batch
        idx += batch_size
        
        if i%5 == 0 :
            print("log : ", avg_cost)
            
    #train cost & acc
    cost, acc = model.evaluate(train_data, train_label, batch_size = batch_size)
    
    train_losses.append(cost)
    train_accs.append(acc)
    
    #valid cost & acc
    v_cost, v_acc = model.evaluate(valid_data, valid_label, batch_size = batch_size)
    
    valid_losses.append(v_cost)
    valid_accs.append(v_acc)
    
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid {:.5f}({:.1f}%)".format(cost, acc*100, v_cost, v_acc*100))
    print(" ")

print("")
print('Learning Finished!')

In [ ]:
print('Accuracy:', model.get_accuracy(test_data, test_label))

In [ ]:
tf.reset_default_graph() 

 <br></br>  <br></br>  <br></br>  <br></br>